In [9]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from scipy.misc import imread
from sklearn.externals import joblib

import random as rand
import numpy as np 
import cv2
import glob
import time

import matplotlib.pyplot as plt
%matplotlib inline

from helpers import convert, show_images
from featuresourcer import FeatureSourcer

import warnings
warnings.filterwarnings('ignore')

In [10]:
sourcer_params = {
  'color_model': 'yuv',                # hls, hsv, yuv, ycrcb
  'bounding_box_size': 64,             #
  'number_of_orientations': 11,        # 6 - 12
  'pixels_per_cell': 16,               # 8, 16
  'cells_per_block': 2,                # 1, 2
  'do_transform_sqrt': True
}

start_frame = cv2.resize(imread("train/insect_pictures/Screenshot 2019-03-07 at 14.50.06.png"),(150, 150))
sourcer = FeatureSourcer(sourcer_params, start_frame)

In [11]:
print("Loading images to memory...")
t_start = time.time()

vehicle_imgs, nonvehicle_imgs = [], []
vehicle_paths = glob.glob('train/insect_pictures/*.png')
nonvehicle_paths = glob.glob('train/non_insect_pictures/*.png')

for path in vehicle_paths: vehicle_imgs.append(cv2.resize(imread(path),(64, 64)))
for path in nonvehicle_paths: nonvehicle_imgs.append(cv2.resize(imread(path),(64, 64)))

vehicle_imgs, nonvehicle_imgs = np.asarray(vehicle_imgs), np.asarray(nonvehicle_imgs)
total_vehicles, total_nonvehicles = vehicle_imgs.shape[0], nonvehicle_imgs.shape[0]

print("... Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print("Insect images shape: ", vehicle_imgs.shape)
print("Non-Insect images shape: ", nonvehicle_imgs.shape)

print("Extracting features... This might take a while...")
t_start = time.time()

vehicles_features, nonvehicles_features = [], []

print("Insect...")
for img in vehicle_imgs:
  vehicles_features.append(sourcer.features(img))
  print('█', end = '')

print()
print("Non-Insect...")
for img in nonvehicle_imgs:
  nonvehicles_features.append(sourcer.features(img))
  print('█', end = '')
    
vehicles_features = np.asarray(vehicles_features)
nonvehicles_features = np.asarray(nonvehicles_features)

print()
print("...Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print("Insect features shape: ", vehicles_features.shape)
print("Non-Insect features shape: ", nonvehicles_features.shape)

print("Scaling features...")
t_start = time.time()

print(vehicles_features.shape)
print(nonvehicles_features.shape)

unscaled_x = np.vstack((vehicles_features, nonvehicles_features)).astype(np.float64)
scaler = StandardScaler().fit(unscaled_x)
x = scaler.transform(unscaled_x)
y = np.hstack((np.ones(total_vehicles), np.zeros(total_nonvehicles)))

print("...Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print(" x shape: ", x.shape, " y shape: ", y.shape)

print("Training classifier...")
t_start = time.time()

print(x.shape,y.shape)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2,
                                                    random_state = rand.randint(1, 100))
svc = LinearSVC()
svc.fit(x_train, y_train)
accuracy = svc.score(x_test, y_test)

print("...Done")
print("Time Taken:", np.round(time.time() - t_start, 2))
print("Accuracy: ", np.round(accuracy, 4))

Loading images to memory...
... Done
Time Taken: 0.46
Insect images shape:  (9, 64, 64, 4)
Non-Insect images shape:  (13, 64, 64, 4)
Extracting features... This might take a while...
Insect...
█████████
Non-Insect...
█████████████
...Done
Time Taken: 0.34
Insect features shape:  (9, 1188)
Non-Insect features shape:  (13, 1188)
Scaling features...
(9, 1188)
(13, 1188)
...Done
Time Taken: 0.01
 x shape:  (22, 1188)  y shape:  (22,)
Training classifier...
(22, 1188) (22,)
...Done
Time Taken: 0.01
Accuracy:  0.4


In [12]:
sourcer = FeatureSourcer(sourcer_params, start_frame)

f1 = sourcer.features(nonvehicle_imgs[rand.randint(0, total_nonvehicles - 1)])
f2 = sourcer.features(vehicle_imgs[rand.randint(0, total_vehicles - 1)])
f3 = sourcer.features(vehicle_imgs[rand.randint(0, total_vehicles - 1)])
f4 = sourcer.features(nonvehicle_imgs[rand.randint(0, total_nonvehicles - 1)])

f = scaler.transform([f1, f2, f3, f4])
print(svc.predict(f))

[1. 1. 1. 0.]


In [13]:
print ("Saving models...")

joblib.dump(svc, 'svc2.pkl')
joblib.dump(scaler, 'scaler2.pkl')

print("...Done")

Saving models...
...Done


In [14]:
print ("Loading models...")

svc = joblib.load('svc2.pkl')
scaler = joblib.load('scaler2.pkl')

print("...Done")

Loading models...
...Done


In [15]:
sourcer = FeatureSourcer(sourcer_params, start_frame)
print(total_nonvehicles)
f1 = sourcer.features(nonvehicle_imgs[rand.randint(0, total_nonvehicles - 1)])
f2 = sourcer.features(vehicle_imgs[rand.randint(0, total_vehicles - 1)])
f3 = sourcer.features(vehicle_imgs[rand.randint(0, total_vehicles - 1)])
f4 = sourcer.features(nonvehicle_imgs[rand.randint(0, total_nonvehicles - 1)])

f = scaler.transform([f1, f2, f3, f4])
print(svc.predict(f))

13
[1. 1. 1. 1.]
